# Immigration Dimensional Modelling
### Data Engineering Capstone Project

#### Project Summary
This is the capstone Data Engineering project about Immigration process focused to Immagrtion from the I94 process. My thoughts about this data was to have a good knowledge about the process and add value to the data through Modelling Dimensionally the data provided in this project as well as the data added and researched by me. All the data together can bring a lot of power in terms of knowledge.

**The final purpose is to know how many immigrants came to the US and know where they come from,the airport they arrived, what type of vise they have, which state they are visiting/staying.**

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import configparser
import os
import datetime as dt
import pandas as pd
import psycopg2
from sql_queries import copy_data, create_sql_queries, drop_sql_queries
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, TimestampType
from pyspark.sql.functions import udf, col, to_timestamp, from_unixtime,monotonically_increasing_id, desc, isnan, when, count
from pyspark.sql.functions import quarter, dayofmonth, weekofyear, month, year, dayofweek

### Step 1: Scope the Project and Gather Data

#### Scope 
A brief definition of I94 Form: *Form I-94, the Arrival-Departure Record Card, is a form used by U.S. Customs and Border Protection (CBP) intended to keep track of the arrival and departure to/from the United States of people who are not United States citizens or lawful permanent residents (with the exception of those who are entering using the Visa Waiver Program or Compact of Free Association, using Border Crossing Cards, re-entering via automatic visa revalidation, or entering temporarily as crew members).*

Starting with the definition above the idea of this project is look at the immigration event and add some useful context to provide a full (or at least a good one) view of the Immigration process filled through **the Form I-94, the Arrival-Departure Record Card.** 

You have a fact named **'FactImmigration'** that describe the Immigration process. And we added the following dimension to give better context to that measure. the dimension are:
* Airport.
* State.
* Visa.
* Time.
* Miscellaneous data about the traveller (The dataset has not sensitive information about the traveller).

Also the Technology used in this project is:
* S3 Bucket.
* Python.
* Apache Spark.
* AWS EMR Cluster.
* Amazon Redshift.

#### Data Architecture of the project:
![alt text](./assets/data_architecture.jpg)


And finally the data used here:
* airport-code_csv.csv
* immigration_data.csv
* visa_type_data.json
* state_descriptions.json
#### Describe and Gather Data 

In [2]:
def create_or_get_spark_session():
    spark = SparkSession \
    .builder \
    .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
    .enableHiveSupport() \
    .getOrCreate()

    return spark

spark = create_or_get_spark_session()

#Print SparkSession
spark

### State dataset overview
This data set is a brief and short dataset thought in mind to be a dimension in the Dimensional Modelling and is focused on US states to analyze the immigration on the differents states. The dataset was extracted from the _**I94_SAS_Labels_Descriptions.SAS**_ file and cleaned via Gsheet formulas. (keep in mind that this kind of dataset are short dataset so we can do it manually directly)

Data Catalog:
* country_code = iso_country code related to a particular state.
* state_code = a state code. In this case they are all from US.
* state_description = full name of the state.


In [3]:
fname_state_data = './data/state_descriptions.json'
us_states_ds = spark.read.option("multiline","true") \
      .json(fname_state_data)
us_states_ds.printSchema()

root
 |-- country_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- state_description: string (nullable = true)



In [4]:
us_states_ds.describe().toPandas()

,summary,country_code,state_code,state_description
0,count,55,55,55
1,mean,None,None,None
2,stddev,None,None,None
3,min,,AK,ALABAMA
4,max,US,WY,WYOMING


In [5]:
us_states_ds.limit(5).toPandas()

,country_code,state_code,state_description
0,US,AL,ALABAMA
1,US,AK,ALASKA
2,US,AZ,ARIZONA
3,US,AR,ARKANSAS
4,US,CA,CALIFORNIA


### Visa dataset Overview
This particular dataset was made from PDF table extracted using "Tabula" a PDF text/table extractor app (you can take a look here: https://tabula.technology/) and I convert the csv to JSON (take a look to the converter: https://www.convertcsv.com/csv-to-json.htm). I cleaned the text with _'\n'_ in a Gsheet because I wanted to add more technologies to not only do everthing automated but also manually cleaned data like Data Analyst. 

Data Catalog:
* **annual_numeric_limit** = the annual numeric limit (or “cap”) for each nonimmigrant and LPR category.
* **description** = a brief explanation of the visa category.
* **initial_duration_of_staya** = the allowed duration of stay in the United States for each nonimmigrant visa category.
* **visa_category** = a list of nonimmigrant (i.e., temporary) visa categories and lawful permanent resident categories.

In [6]:
fname_visa_data = './data/visa_type_data.json'
visa_ds = spark.read.option("multiline","true") \
      .json(fname_visa_data)
visa_ds.printSchema()

root
 |-- annual_numeric_limit: string (nullable = true)
 |-- description: string (nullable = true)
 |-- initial_duration_of_staya: string (nullable = true)
 |-- visa_category: string (nullable = true)



In [7]:
visa_ds.describe().toPandas()

,summary,annual_numeric_limit,description,initial_duration_of_staya,visa_category
0,count,81,81,81,81
1,mean,55.166666666666664,None,None,None
2,stddev,75.79028081928887,None,None,None
3,min,0,"Adult or minor child of T-2, T-3, T-4, or T-5",Alien trainee: up to\ntwo years\nSpecial educa...,A1
4,max,"Specialty occupation or fashion model: 65,000,...",Witness or informant in terrorism matter,Valid for four months; must marry within 90 da...,V2


In [8]:
visa_ds.limit(5).toPandas()

,annual_numeric_limit,description,initial_duration_of_staya,visa_category
0,None,"Ambassador, public minister, career diplomat, ...",Duration of assignment,A1
1,None,"Other foreign government official or employee,...",Duration of assignment,A2
2,None,"Attendant or personal employee of A-1/A-2, and...",Up to three years,A3
3,None,Visitor for business,Six months to one year,B1
4,None,Visitor for pleasure,Six months to one year,B2


### Airport Dataset Overview
“data/airport-codes.csv” contains the list of all airport codes, the attributes are identified in datapackage description. Some of the columns contain attributes identifying airport locations, other codes (IATA, local if exist) that are relevant to identification of an airport. The dataset is from: https://datahub.io/core/airport-codes. There is a couple of information to take a look

Data Catalog:
* **type** = airport type.
* **name** = name of the airport.
* **elevation_ft** = elevation of the airport related to the ocean.
* **continent** = continent where the airport belongs.
* **iso_country** = country iso code.
* **iso_region** = iso region code.
* **municipality** = municipality name where the airport belongs.
* **gps_code** = gps code of the airport.
* **iata_code** = a three-letter geocode designating many airports and metropolitan areas around the world.
* **local_code** = local code of the airport related to the country it belongs.
* **coordinates** = coordinates where the airport it is.

### Immigration Dataset Overview
This particular dataset is focused on the 2016 Form I-94 dataset that keep track of the immigrants that came to US and relevant information about them. Obviously there is no sensitive data about this wonderful people, only the necessary for the study case. 

**_This is going to be our fact table, the measure here is the immigration event itself._** This fact table is a special kind of fact table. It's a Factless fact table that is a fact table wih not measures because here we are capturing the event of the immigration with the Form I-94. This event establishes the relationship among the dimensions declared above. I mean, the existence of the relationship itself is the fact.

Data Catalog:
* **cicid** = is a unique number for the immigrants.
* **i94yr** = 4 digit year.
* **i94mon** = numeric month.
* **i94cit** = 3 digit code of origin city.
* **i94res** = country from where the immigrant has travelled.
* **i94port** = 3 character code of destination port.
* **arrdate** = arrival date in USA. It is in SAS data numeric field.
* **i94mode** = how the traveller came to US.
* **i94addr** = state where the immigrants reside in USA.
* **depdate** = is the Departure date from the USA. It is in SAS data numeric field.
* **i94bir** = Age of respondent in Years.
* **i94visa** = visa codes collapesed intro three categories.
* **count** = used for summary statistics.
* **dtadfile** = Character date field - Date added to I-94 Files - CIC does not use.
* **visapost** = department of state where the visa was issued - CIC does not use.
* **occup** = occupation that will be perfomed in USA - CIC does not use.
* **entdepa** = arrival flag - admitted or paroled into the USA - CIC does not use.
* **entdepd** = departure flag - departed, lost I-94 or is deceased - CIC does not use.
* **entdepu** = update flag - either apprehended, overstayed, adjusted to perm residence - CIC does not use.
* **matflag** = match flag - match of arrival and departure records.
* **biryear** = 4 digit year of birth.
* **dtaddto** = character date field - Date to which admitted to USA. Allowed to stay unitl - CIC does not use.
* **gender** = gender of the immigrant.
* **insnum** = INS number.
* **airline** = airline used to arrive in U.S.
* **admnum** = admission number.
* **fltno** = flight number of airline used to arrive in U.S.
* **visatype** = class of admission legally admitting the non-immigrant to temporarily stay in U.S.

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
##### Cleaning Steps
Document steps necessary to clean the data

### Immigration dataset

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [9]:
fname_imm = './sas_data/'
immigration_ds = spark.read.parquet(fname_imm)
immigration_ds.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

##### Brief descriptive statistics about the columns.

In [10]:
immigration_ds.describe().toPandas()

,summary,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,count,3096313,3096313,3096313,3096313,3096313,3096313,3096313,3096074,2943721,...,392,2957884,3095511,3095836,2682044,113708,3012686,3096313,3076764,3096313
1,mean,3078651.879075533,2016.0,4.0,304.9069344733559,303.28381949757664,None,20559.84854179794,1.0736897761487614,51.652482269503544,...,None,None,1974.2323855415148,8291120.333841449,None,4131.050016327899,59.477601493233784,7.082885011150484E10,1360.2463696420555,None
2,stddev,1763278.0997499449,0.0,0.0,210.02688853063205,208.58321292789535,None,8.777339475317723,0.5158963131657106,42.97906231370983,...,None,None,17.420260534589556,1656502.4244925722,None,8821.743471773654,172.6333995206175,2.2154415947558968E10,5852.676345633695,None
3,min,6.0,2016.0,4.0,101.0,101.0,5KE,20545.0,1.0,..,...,U,M,1902.0,/ 183D,F,0,*FF,0.0,00000,B1
4,max,6102785.0,2016.0,4.0,999.0,760.0,YSL,20574.0,9.0,ZU,...,Y,M,2019.0,D/S,X,YM0167,ZZ,9.991556593E10,ZZZ,WT


###### Looking for NaN/Null values

In [11]:
immigration_ds.select([count(when(col(c).isNull(), c)).alias(c) for c in immigration_ds.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

we have a few ways to clean or manage those null above. 
- [x]  Here for mayority of the columns I'm going to drop it because they are not useful for the scope of this project.
- [x]  gender table I want to be more inclusive so those who does not have gender is going to be replaced as non-binary/other/prefer_not_to_respond.
- [x]  i94addr I'm going to classify the null/NaN value as multiple states (because they are travelling around the country).
- [x]  depdate the null/NaN values are meaningful because gives you the idea that they are still in the US.

Dropping duplicates rows.

###### Rename the columns for better understanding, also I wanted to focus the data only with people tha came from Air and landed in an port as well (filtering by i94mode == 1).

In [12]:
immigration_renamed = immigration_ds.select(
    col('cicid').alias('imm_id'),
    col('i94yr').alias('year'),
    col('i94mon').alias('month'),
    col('i94port').alias('port'),
    col('i94res').alias('country_origin'),
    col('arrdate').alias('arrival_date'),
    col('depdate').alias('departure_date'),
    col('i94mode').alias('immigration_type'),
    col('i94addr').alias('temp_state_residence'),
    col('gender').alias('gender'),
    col('i94visa').alias('visa_type'),
    col('visatype').alias('visa_category')
).where(immigration_ds['i94mode'] == 1)

###### fullfilling the null values on Gender column

In [13]:
gender_non_binary = udf(lambda x: 'X' if x == None or x == '' else x)

In [14]:
immigration_renamed.withColumn('new_gender_col', gender_non_binary(immigration_renamed['gender'])) \
                    .select('new_gender_col') \
                    .distinct() \
                    .toPandas()

,new_gender_col
0,F
1,M
2,U
3,X


In [16]:
immigration_renamed.withColumn('new_gender_col', gender_non_binary(immigration_renamed['gender'])) \
                    .select('new_gender_col') \
                    .groupBy('new_gender_col') \
                    .count() \
                    .show()

+--------------+-------+
|new_gender_col|  count|
+--------------+-------+
|             F|1255002|
|             M|1326478|
|             U|    117|
|             X| 412908|
+--------------+-------+



###### filling the null values in i94addr/temp_state_residence. 

In [17]:
multiple_states_addr = udf(lambda x: 'multiple_states' if x == None or x == '' else x)

In [18]:
immigration_renamed.withColumn('new_temp_state_col', multiple_states_addr(immigration_renamed['temp_state_residence'])) \
                    .select('new_temp_state_col') \
                    .distinct() \
                    .show()

+------------------+
|new_temp_state_col|
+------------------+
|                .N|
|                RG|
|                YH|
|                RF|
|                CI|
|                FT|
|                TC|
|                SC|
|                AZ|
|                IC|
|                FI|
|                PU|
|                UA|
|                EA|
|                NS|
|                KI|
|                RO|
|                PI|
|                SL|
|                LA|
+------------------+
only showing top 20 rows



###### Drop duplicates in Immigration dataset.

In [19]:
immigration_ds.count()

3096313

In [20]:
immigration_ds.dropDuplicates().count()

3096313

###### Converting SAS Numeric Date to date with UDF for arrival_date and departure_date

In [21]:
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(float(x))).isoformat() if x else None)

In [22]:
immigration_cleaned = immigration_renamed \
            .withColumn('arrival_date_sas', immigration_renamed['arrival_date']) \
            .withColumn('departure_date_sas', immigration_renamed['departure_date']) \
            .withColumn('arrival_date_f', get_date(immigration_renamed['arrival_date'])) \
            .withColumn('departure_date_f', get_date(immigration_renamed['departure_date'])) \
            .drop(col('arrival_date')) \
            .drop(col('departure_date'))

In [23]:
immigration_cleaned.limit(5).toPandas()

,imm_id,year,month,port,country_origin,immigration_type,temp_state_residence,gender,visa_type,visa_category,arrival_date_sas,departure_date_sas,arrival_date_f,departure_date_f
0,5748517.0,2016.0,4.0,LOS,438.0,1.0,CA,F,1.0,B1,20574.0,20582.0,2016-04-30,2016-05-08
1,5748518.0,2016.0,4.0,LOS,438.0,1.0,NV,F,1.0,B1,20574.0,20591.0,2016-04-30,2016-05-17
2,5748519.0,2016.0,4.0,LOS,438.0,1.0,WA,M,1.0,B1,20574.0,20582.0,2016-04-30,2016-05-08
3,5748520.0,2016.0,4.0,LOS,438.0,1.0,WA,F,1.0,B1,20574.0,20588.0,2016-04-30,2016-05-14
4,5748521.0,2016.0,4.0,LOS,438.0,1.0,WA,M,1.0,B1,20574.0,20588.0,2016-04-30,2016-05-14


In [24]:
df_time = immigration_cleaned.withColumn('day',        dayofmonth(immigration_cleaned['arrival_date_f'])) \
                            .withColumn('week',       weekofyear(immigration_cleaned['arrival_date_f'])) \
                            .withColumn('month',      month(immigration_cleaned['arrival_date_f'])) \
                            .withColumn('year',       year(immigration_cleaned['arrival_date_f'])) \
                            .withColumn('weekday',    dayofweek(immigration_cleaned['arrival_date_f'])) \
                            .withColumn('quarter',     quarter(immigration_cleaned['arrival_date_f'])) \
                            .select(col('arrival_date_sas').alias('sas_date_id'), 'arrival_date_f','day','week', 'month', 'year', 'weekday', 'quarter') \
                            .dropDuplicates()
df_time.limit(10).toPandas()

,sas_date_id,arrival_date_f,day,week,month,year,weekday,quarter
0,20550.0,2016-04-06,6,14,4,2016,4,2
1,20563.0,2016-04-19,19,16,4,2016,3,2
2,20573.0,2016-04-29,29,17,4,2016,6,2
3,20557.0,2016-04-13,13,15,4,2016,4,2
4,20552.0,2016-04-08,8,14,4,2016,6,2
5,20548.0,2016-04-04,4,14,4,2016,2,2
6,20561.0,2016-04-17,17,15,4,2016,1,2
7,20567.0,2016-04-23,23,16,4,2016,7,2
8,20574.0,2016-04-30,30,17,4,2016,7,2
9,20569.0,2016-04-25,25,17,4,2016,2,2


In [26]:
df_time.count()

30

### Airport Dataset.

In [27]:
fname_imm = './data/airport-codes_csv.csv'
airport_ds = spark.read.csv(fname_imm, header=True, inferSchema=True)
airport_ds.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



###### Brief descriptive statistics about the columns.

In [28]:
airport_ds.describe().toPandas()

,summary,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,count,55075,55075,55075,48069,55075,55075,55075,49399,41030,9189,28686,55075
1,mean,2.3873375337777779E8,None,None,1240.7896773388254,None,None,None,None,2.1920446610204083E8,0.0,8.580556178571428E7,None
2,stddev,9.492375382267495E8,None,None,1602.3634593484142,None,None,None,None,9.1123224377024E8,0.0,5.747026415216715E8,None
3,min,00A,balloonport,"""""""Der Dingel"""" Airfield""",-1266,AF,AD,AD-04,'S Gravenvoeren,0000,-,-,"-0.004722000099718571, 9.425000190734863"
4,max,spgl,small_airport,Çá¸¾á¸á¸ á¸®á¸Ç{+91-9680118734} GiRLFRieNd...,22000,SA,ZZ,ZZ-U-A,Å½ocene,ZYYY,ZZV,ZZV,"99.9555969238, 8.47115039825"


###### Looking at the null/Nan values in the columns.

In [29]:
airport_ds.select([count(when(col(c).isNull(), c)).alias(c) for c in airport_ds.columns]).show()

+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|ident|type|name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|    0|   0|   0|        7006|        0|          0|         0|        5676|   14045|    45886|     26389|          0|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+



As you can see, there is a lot of null values in local_code, gps_code, iata_code, municipality. Filtering by iso_country = 'US'. Also we can ignore him as well and joining local_code from airport_ds with immigration_ds we can only extract those who are valid/have a local_code.

In [30]:
aiport_filtered_ds = airport_ds.filter("type in ('large_airport','medium_airport','small_airport') AND iso_country = 'US'")

In [31]:
aiport_filtered_ds.select('type').distinct().show()

+--------------+
|          type|
+--------------+
| large_airport|
|medium_airport|
| small_airport|
+--------------+



In [32]:
aiport_filtered_ds.describe().toPandas()

,summary,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,count,14582,14582,14582,14519,14582,14582,14582,14532,14183,1865,14383,14582
1,mean,1.68575467725E8,None,None,1252.9349817480543,None,None,None,None,1.6441509092682928E8,None,7.022043930927835E7,None
2,stddev,8.018295980395926E8,None,None,1454.7597367351611,None,None,None,None,7.921911984023423E8,None,5.177553655730362E8,None
3,min,00AA,large_airport,"""Fly """"N"""" K Airport""",-210,NA,US,US-AK,Abbeville,00AA,AAF,00AA,"-100, 29.911300659179688"
4,max,ZNC,small_airport,hln,9927,NA,US,US-WY,Zwolle,ZNC,ZZV,ZZV,"8.4375, 11.523088"


In [33]:
aiport_filtered_ds.select([count(when(col(c).isNull(), c)).alias(c) for c in aiport_filtered_ds.columns]).show()

+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|ident|type|name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|    0|   0|   0|          63|        0|          0|         0|          50|     399|    12717|       199|          0|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+



In [34]:
airport_cleaned = aiport_filtered_ds.dropDuplicates()

In [35]:
airport_renamed = airport_cleaned.select(
        col('local_code').alias('airport_local_code'),
        col('name').alias('airport_name'),
        col('type').alias('airport_type'),
        col('iso_country'),
        col('continent'),
        col('municipality')
)
    
airport_cleaned_na = airport_renamed.na.drop(subset=["airport_local_code"]) 

airport_cleaned_na.select([count(when(col(c).isNull(), c)).alias(c) for c in airport_cleaned_na.columns]).show()

+------------------+------------+------------+-----------+---------+------------+
|airport_local_code|airport_name|airport_type|iso_country|continent|municipality|
+------------------+------------+------------+-----------+---------+------------+
|                 0|           0|           0|          0|        0|          12|
+------------------+------------+------------+-----------+---------+------------+



In [36]:
 airport_cleaned_na.count()

14383

#### State Dataset and Visa Type dataset.
For those dataset they are already cleaned manually. So we just need to map the dataset to the data model.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

Here is a ER diagram to show you the Dimensional Modelling that I desgined.
 	![alt text](./assets/db_diagram-db_diagram.jpg)

I decided to follow the Dimensional Modelling concepts because I wanted to know how many immigrants has travelled to US but also wanted to add more context and insight. So I desgined a factless fact table that measure the event of travel to US as immigrant and expose the relations between differents dimensions like Time, Airport, State, Visa and So on.

I decided to leave some traveller data as it is in the Fact table because it's known as Junk_data where it hasn't all the relevant data to conform an a separate dimension (of course, you can set a miscellaneous dimnesion and create a concatenate key from all those values and move the columns to the new column.)
#### 3.2 Mapping Out Data Pipelines

1. Read data from S3 Bucket (staging folder) or read from local .
2. Perform data cleaning.
3. Create Data Model.
4. Write it down as parquet format into S3 Bucket.
5. Load into AWS Redshift.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### It's time to code and build the data pipeline. Here, first of all we are going to create the step to get the data, process it and load it through the function coded below. The idea is execute it through python or scheduled with AWS EMR jobs to execute per intervals
![alt text](./assets/db_diagram-ETL_Process.jpg)

##### *  _create_spark_session_ is the responsible to create a spark session and config all the packages required to accsess the data and process it. And _create_conn_cur_postgres_ create the connection and the cursor to execute the query and load it to redshift.


#### ...........Troubleshooting...........
If you are having problem to execute the process.. There is a issue releted to _**"java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found"**_ follow next steps:
1. Go to SPARK_HOME directory.
2. Move it to the 'jars' folder.
3. Execute the following comands:
    * sudo wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.11.30/aws-java-sdk-1.11.30.jar
    * sudo wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar
4. After that refresh the workspace and execute the process again.

For more details: https://stackoverflow.com/questions/58415928/spark-s3-error-java-lang-classnotfoundexception-class-org-apache-hadoop-f

In [2]:
# Write code here
config = configparser.ConfigParser()
config.read('dwh.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


def create_spark_sessions3():
    """
    Description: This function is responsible to create or gate an SparkSession and use the aws credentials
    to connect to the data source s3 to consume and load data.
    
    Arguments:
        None
        
    Return:
        spark: SparkSession object
    """
    spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.3") \
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID']) \
    .config("spark.hadoop.fs.s3a.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY']) \
    .enableHiveSupport() \
    .getOrCreate()

    return spark

def create_conn_cur_postgres(config):
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    return conn, cur

##### process_immigration_data_and_time_dimension read the data from S3 Bucket, transform it, and finally load it to S3 bucket TEMP

In [3]:
def process_immigration_data_and_time_dimension(spark, input_data, output_data):
    """
    Description: This function is responsible to process immigration data through reading data from s3 bucket/local data, then
    processing it with apache spark dataframe to create immigration and time table. Finally write it down as parquet file into S3
    Arguments:
        - spark: SparkSession Object to process the data
        - input_data: String with the s3 URL to get the data
        - output_data: String with the s3 URL to write the data
    Return:
        None
    """
    
    # read immigration_data file
    df = spark.read.parquet(input_data)
    
    print('Starting Data Cleainng process...')
    
    print('* Renaming columns from immigration dataset.. \n')
    # extract columns to create immigration table
    df_renamed = df.select(
        col('cicid').alias('imm_id'),
        col('i94yr').alias('year'),
        col('i94mon').alias('month'),
        col('i94port').alias('port'),
        col('i94res').alias('country_origin'),
        col('arrdate').alias('arrival_date'),
        col('depdate').alias('departure_date'),
        col('i94mode').alias('immigration_type'),
        col('i94addr').alias('temp_state_residence'),
        col('gender').alias('gender'),
        col('i94visa').alias('visa_type'),
        col('visatype').alias('visa_category')
    ).where(df['i94mode'] == 1)
    
    
    print('* Creating UDFs to replaces nulls values.. \n')
    gender_non_binary = udf(lambda x: 'X' if x == None or x == '' else x)
    
    df_gender_replaced = df_renamed.withColumn('gender', gender_non_binary(df_renamed['gender']))
    
    multiple_states_addr = udf(lambda x: 'multiple_states' if x == None or x == '' else x)
    
    df_temp_state_replaced = df_gender_replaced.withColumn('temp_state_residence', multiple_states_addr(df_gender_replaced['temp_state_residence']))
    
    
    print('* Converting SAS numeric date to Date.. \n')
    get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(float(x))).isoformat() if x else None)
    
    df_date_formatted = df_temp_state_replaced \
            .withColumn('arrival_date_sas', df_temp_state_replaced['arrival_date']) \
            .withColumn('departure_date_sas', df_temp_state_replaced['departure_date']) \
            .withColumn('arrival_date_f', get_date(df_temp_state_replaced['arrival_date'])) \
            .withColumn('departure_date_f', get_date(df_temp_state_replaced['departure_date'])) \
            .drop(col('arrival_date')) \
            .drop(col('departure_date')) \
            .drop(col('year')) \
            .drop(col('month')) \
            .dropDuplicates()    
    
    print('* Extracting the columns to create DimTime dataframe.. \n')
    df_time = df_date_formatted.withColumn('day',        dayofmonth(df_date_formatted['arrival_date_f'])) \
                            .withColumn('week',       weekofyear(df_date_formatted['arrival_date_f'])) \
                            .withColumn('month',      month(df_date_formatted['arrival_date_f'])) \
                            .withColumn('year',       year(df_date_formatted['arrival_date_f'])) \
                            .withColumn('weekday',    dayofweek(df_date_formatted['arrival_date_f'])) \
                            .withColumn('quarter',     quarter(df_date_formatted['arrival_date_f'])) \
                            .select(col('arrival_date_sas').alias('sas_date_id'), 'arrival_date_f','day','week', 'month', 'year', 'weekday', 'quarter') \
                            .dropDuplicates()
   
    
    print('* Dropping unnecesary columns.. \n')
    de_date_formatted_v2 = df_date_formatted.drop(col('arrival_date_f')) \
                                            .drop(col('departure_date_f'))
    
    time_table_path = os.path.join(output_data, 'DimTime/DimTime.parquet')
    immigration_table_path = os.path.join(output_data, 'FactImmigration/FactImmigraton.parquet')
    
    print('Loading in DimTime to "{}" and FactImmigration to "{}" in PARQUET format.. \n'.format(time_table_path, immigration_table_path))
    
    de_date_formatted_v2.write.mode('overwrite').parquet(immigration_table_path)
    
    df_time.write.mode('overwrite').parquet(time_table_path)
    
    print('Step Sucessfully executed.. \n')
    
    
def process_airport_dimension(spark, input_data, output_data):
    """
    Description: This function is responsible to process airport data through reading data from s3 bucket, then
    processing it with apache spark dataframe to create aiport table. Finally write it down as parquet file into S3
    Arguments:
        - spark: SparkSession Object to process the data
        - input_data: String with the s3 URL to get the data
        - output_data: String with the s3 URL to write the data
    Return:
        None
    """ 
    print('* Reading airport data from S3 bucket.. \n')
    df = spark.read.csv(input_data, header=True, inferSchema=True)
    
    print('* Filtering only aiports and iso_country = "US".. \n')
    aiport_filtered_df = df.filter("type in ('large_airport','medium_airport','small_airport') AND iso_country = 'US'")
    
    airport_cleaned = aiport_filtered_df.dropDuplicates()
    
    print('* Renaming columns and extracting them.. \n')
    airport_renamed = airport_cleaned.select(
        col('local_code').alias('airport_local_code'),
        col('name').alias('airport_name'),
        col('type').alias('airport_type'),
        col('iso_country'),
        col('continent'),
        col('municipality')
    )
    
    airport_cleaned_na = airport_renamed.na.drop(subset=["airport_local_code"]) 
    
    airport_table_path = os.path.join(output_data, 'DimAirport/DimAirport.parquet')
    print(f'* Loading the Airport DataFrame to S3 Bucket "{airport_table_path}".. \n')
    airport_cleaned_na.write.mode('overwrite').parquet(airport_table_path)
    

def process_dimensions(spark, input_data, output_data):
    """
    Description: This function is responsible to process airport data through reading data from s3 bucket, then
    processing it with apache spark dataframe to create aiport table. Finally write it down as parquet file into S3
    Arguments:
        - spark: SparkSession Object to process the data
        - input_data: String with the s3 URL to get the data
        - output_data: String with the s3 URL to write the data
    Return:
        None
    """
    country_filepath = os.path.join(input_data, 'imm_i94_country_code.csv')
    print(f'* Loading Country Dataset from S3 Bucket "{country_filepath}".. \n')
    country_ds = spark.read.csv(country_filepath, header=True, inferSchema=True)
    
    country_table_path = os.path.join(output_data, 'DimCountry/DimCountry.parquet')
    print(f'* Writing DimCountry to S3 Bucket "{country_table_path}".. \n')
    country_ds.write.mode('overwrite').parquet(country_table_path)
    
    
    state_filepath = os.path.join(input_data, 'state_descriptions.json')
    print(f'* Loading State Dataset from S3 Bucket "{state_filepath}".. \n')
    state_ds = spark.read.option("multiline","true").json(state_filepath)
    
    
    state_table_path = os.path.join(output_data, 'DimState/DimState.parquet')
    print(f'* Writing DimState to S3 Bucket "{state_table_path}".. \n')
    state_ds.write.mode('overwrite').parquet(state_table_path)
    
    visa_filepath = os.path.join(input_data, 'visa_type_data.json')
    print(f'* Loading Visa Dataset to S3 Bucket "{visa_filepath}".. \n')
    visa_ds = spark.read.option("multiline","true").json(visa_filepath)
    
    
    visa_table_path = os.path.join(output_data, 'DimVisa/DimVisa.parquet')
    print(f'* Writting DimVisa to S3 Bucket "{visa_table_path}"')
    visa_ds.write.mode('overwrite').parquet(visa_table_path)
    

In [4]:
def drop_tables(cur, conn):
    for query in drop_sql_queries:
        print('Dropping tables...')
        print('Executing query: {}'.format(query))
        cur.execute(query)
        conn.commit()

def create_tables(cur, conn):
    for query in create_sql_queries:
        print('Creating tables..')
        print('Executing query: {}'.format(query))
        cur.execute(query)
        conn.commit()

def copy_from_s3_to_redshift(cur, conn, tables, output_data):
    for table in tables:
        query = copy_data.format(
            table=table,
            s3_bucket = os.path.join(output_data, table),
            iam_role = config['IAM_ROLE']['ARN']
        )
        print('Executing query: {}'.format(query))
        cur.execute(query)
        conn.commit()

In [5]:
#Setting up initial variable to execute the ETL PROCESS..
input_data = './sas_data/'
output_data = 's3a://capstone-project-data-lake/data_cleanned/'
s3_input_data = 's3a://capstone-project-data-lake/staging/'

tables = ['FactImmigration', 'DimTime', 'DimAirport', 'DimCountry', 'DimState', 'DimVisa']

data_check_queries = [{'check_query': 'SELECT COUNT(1) FROM factimmigration;','expected_result': 2994505},
                      {'check_query': 'SELECT COUNT(1) FROM DimTime;','expected_result': 30},
                      {'check_query': 'SELECT COUNT(1) FROM DimAirport;','expected_result': 14383},
                      {'check_query': 'SELECT COUNT(1) FROM DimState;','expected_result': 55},
                      {'check_query': 'SELECT COUNT(1) FROM DimCountry;','expected_result': 289},
                      {'check_query': 'SELECT COUNT(1) FROM DimVisa;','expected_result': 81}]


# Creating connection and cursor to query AWS Redshift..
conn, cur = create_conn_cur_postgres(config)

# Creating the SparkSession to read and process data from S3..
s3_spark = create_spark_sessions3()

# ...........Executing ETL Process...........
process_immigration_data_and_time_dimension(s3_spark, input_data, output_data)

process_airport_dimension(s3_spark, s3_input_data, output_data)

Starting Data Cleainng process...
* Renaming columns from immigration dataset.. 

* Creating UDFs to replaces nulls values.. 

* Converting SAS numeric date to Date.. 

* Extracting the columns to create DimTime dataframe.. 

* Dropping unnecesary columns.. 

Loading in DimTime to "s3a://capstone-project-data-lake/data_cleanned/DimTime/DimTime.parquet" and FactImmigration to "s3a://capstone-project-data-lake/data_cleanned/FactImmigration/FactImmigraton.parquet" in PARQUET format.. 

Step Sucessfully executed.. 

* Reading airport data from S3 bucket.. 

* Filtering only aiports and iso_country = "US".. 

* Renaming columns and extracting them.. 

* Loading the Airport DataFrame to S3 Bucket "s3a://capstone-project-data-lake/data_cleanned/DimAirport/DimAirport.parquet".. 



In [6]:
process_dimensions(s3_spark, s3_input_data, output_data)

* Loading Country Dataset from S3 Bucket "s3a://capstone-project-data-lake/staging/imm_i94_country_code.csv".. 

* Writing DimCountry to S3 Bucket "s3a://capstone-project-data-lake/data_cleanned/DimCountry/DimCountry.parquet".. 

* Loading State Dataset from S3 Bucket "s3a://capstone-project-data-lake/staging/state_descriptions.json".. 

* Writing DimState to S3 Bucket "s3a://capstone-project-data-lake/data_cleanned/DimState/DimState.parquet".. 

* Loading Visa Dataset to S3 Bucket "s3a://capstone-project-data-lake/staging/visa_type_data.json".. 

* Writting DimVisa to S3 Bucket "s3a://capstone-project-data-lake/data_cleanned/DimVisa/DimVisa.parquet"


In [8]:
conn, cur = create_conn_cur_postgres(config)

In [9]:
drop_tables(cur, conn)

create_tables(cur, conn)

Dropping tables...
Executing query: DROP TABLE IF EXISTS FactImmigration;
Dropping tables...
Executing query: DROP TABLE IF EXISTS DimTime;
Dropping tables...
Executing query: DROP TABLE IF EXISTS DimAirport;
Dropping tables...
Executing query: DROP TABLE IF EXISTS DimCountry;
Dropping tables...
Executing query: DROP TABLE IF EXISTS DimState;
Dropping tables...
Executing query: DROP TABLE IF EXISTS DimVisa;
Creating tables..
Executing query: 
    CREATE TABLE IF NOT EXISTS FactImmigration(
        imm_id					DOUBLE PRECISION PRIMARY KEY,
        port					VARCHAR(250),
        country_origin			DOUBLE PRECISION,
        immigration_type		DOUBLE PRECISION,
        temp_state_residence	VARCHAR(15),
        gender					VARCHAR(10),
        visa_type				DOUBLE PRECISION,
        visa_category			VARCHAR(10),
        arrival_date_sas			DOUBLE PRECISION,
        departure_date_sas			DOUBLE PRECISION

    );

Creating tables..
Executing query: 
    CREATE TABLE IF NOT EXISTS DimTime (
        sas

In [10]:
copy_from_s3_to_redshift(cur, conn, tables, 's3://capstone-project-data-lake/data_cleanned/')

Executing query: 
    COPY FactImmigration
    FROM 's3://capstone-project-data-lake/data_cleanned/FactImmigration'
    IAM_ROLE 'arn:aws:iam::612801625317:role/dwhRole'
    FORMAT AS PARQUET;

Executing query: 
    COPY DimTime
    FROM 's3://capstone-project-data-lake/data_cleanned/DimTime'
    IAM_ROLE 'arn:aws:iam::612801625317:role/dwhRole'
    FORMAT AS PARQUET;

Executing query: 
    COPY DimAirport
    FROM 's3://capstone-project-data-lake/data_cleanned/DimAirport'
    IAM_ROLE 'arn:aws:iam::612801625317:role/dwhRole'
    FORMAT AS PARQUET;

Executing query: 
    COPY DimCountry
    FROM 's3://capstone-project-data-lake/data_cleanned/DimCountry'
    IAM_ROLE 'arn:aws:iam::612801625317:role/dwhRole'
    FORMAT AS PARQUET;

Executing query: 
    COPY DimState
    FROM 's3://capstone-project-data-lake/data_cleanned/DimState'
    IAM_ROLE 'arn:aws:iam::612801625317:role/dwhRole'
    FORMAT AS PARQUET;

Executing query: 
    COPY DimVisa
    FROM 's3://capstone-project-data-lake/dat

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [29]:
# Perform quality checks here

def data_quality_check(cur, conn, queries):
    """
    Description: This function is responsible to check the quality of the data through reading data from AWS Redshift, then
    check if the expected result is the same as the result from the query. Finally print if the data quality checks failed or passed.
    Arguments:
        - cur: Cursor Object.
        - conn: Connection Object.
        - queries: Dictionary with sql queries and expected result.
    Return:
        None
    """ 
    failed_checks = 0
    failed_queries = []
    
    for check in queries:
        sql = check.get('check_query')
        exp_result = check.get('expected_result')
                
        cur.execute(sql)
        result = cur.fetchone()
        
        if result[0] != exp_result:
            print(f'result:{result[0]} - Expected Result:{exp_result}')
            failed_checks+=1
            failed_queries.append(sql)
                
                
    if failed_checks > 0:
        print(f"{failed_checks} data quality checks failed.")
        print(failed_queries)
        raise ValueError('Data Quality Check failed.')
    else:
        print('Data Quality Check Passed :)')

In [30]:
data_quality_check(cur, conn, data_check_queries)

result:(2994505,)
result:(30,)
result:(14383,)
result:(55,)
result:(289,)
result:(81,)
Data Quality Check Passed :)


#### Constraint in the database side.

In [ ]:
CREATE TABLE IF NOT EXISTS FactImmigration(
	imm_id					DOUBLE PRIMARY KEY,
	port					VARCHAR(250) NOT NULL,
	country_origin			VARCHAR(250),
	arrival_date			INT NOT NULL,
	departure_date			INT,
	immigration_type		INT,
	temp_state_residence	VARCHAR(15) NOT NULL,
	gender					VARCHAR(10) CHECK( gender IN ('F','M','X','U') ),
	visa_type				VARCHAR(15),
	visa_category			VARCHAR(10) NOT NULL
);


CREATE TABLE IF NOT EXISTS DimTime (
	sas_date_id			DOUBLE PRIMARY KEY,
	day					INT NOT NULL,
	week				INT NOT NULL,
	month				INT NOT NULL,
	year				INT NOT NULL,
	quarter				INT CHECK ( quarter >=1 and quarter <=4 ),
	arrival_date_f		DATE NOT NULL
);

CREATE TABLE IF NOT EXISTS DimAirport (
	local_code		VARCHAR(100) PRIMARY KEY,
	airport_name	VARCHAR(250) NOT NULL,
	airport_type 	VARCHAR(100) NOT NULL,
	iso_country		VARCHAR(10) NOT NULL,
	continent		VARCHAR(30),
	municipality	varchar(50)
);

CREATE TABLE IF NOT EXISTS DimCountry (
	country_code	VARCHAR(50) PRIMARY KEY,
	country_name	VARCHAR(150) NOT NULL
);

CREATE TABLE IF NOT EXISTS DimState (
	state_code			VARCHAR(10) PRIMARY KEY,
	state_description	VARCHAR(100) NOT NULL,
	country_code		VARCHAR(10)
);

CREATE TABLE IF NOT EXISTS DimVisa (
	visa_category				VARCHAR(50) PRIMARY KEY,
	description					TEXT,
	initial_duration_of_staya	TEXT,
	annual_numeric_limit		VARCHAR(250)
);



In [ ]:
conn.close()

#### 4.3 Data dictionary 

### Immigration Dataset Overview

Data Catalog:
* **cicid** = is a unique number for the immigrants.
* **i94yr** = 4 digit year.
* **i94mon** = numeric month.
* **i94cit** = 3 digit code of origin city.
* **i94res** = country from where the immigrant has travelled.
* **i94port** = 3 character code of destination port.
* **arrdate** = arrival date in USA. It is in SAS data numeric field.
* **i94mode** = how the traveller came to US.
* **i94addr** = state where the immigrants reside in USA.
* **depdate** = is the Departure date from the USA. It is in SAS data numeric field.
* **i94bir** = Age of respondent in Years.
* **i94visa** = visa codes collapesed intro three categories.
* **count** = used for summary statistics.
* **dtadfile** = Character date field - Date added to I-94 Files - CIC does not use.
* **visapost** = department of state where the visa was issued - CIC does not use.
* **occup** = occupation that will be perfomed in USA - CIC does not use.
* **entdepa** = arrival flag - admitted or paroled into the USA - CIC does not use.
* **entdepd** = departure flag - departed, lost I-94 or is deceased - CIC does not use.
* **entdepu** = update flag - either apprehended, overstayed, adjusted to perm residence - CIC does not use.
* **matflag** = match flag - match of arrival and departure records.
* **biryear** = 4 digit year of birth.
* **dtaddto** = character date field - Date to which admitted to USA. Allowed to stay unitl - CIC does not use.
* **gender** = gender of the immigrant.
* **insnum** = INS number.
* **airline** = airline used to arrive in U.S.
* **admnum** = admission number.
* **fltno** = flight number of airline used to arrive in U.S.
* **visatype** = class of admission legally admitting the non-immigrant to temporarily stay in U.S.

### State dataset overview

Data Catalog:
* **country_code** = iso_country code related to a particular state.
* **state_code** = a state code. In this case they are all from US.
* **state_description** = full name of the state.


### Visa dataset Overview

Data Catalog:
* **annual_numeric_limit** = the annual numeric limit (or “cap”) for each nonimmigrant and LPR category.
* **description** = a brief explanation of the visa category.
* **initial_duration_of_staya** = the allowed duration of stay in the United States for each nonimmigrant visa category.
* **visa_category** = a list of nonimmigrant (i.e., temporary) visa categories and lawful permanent resident categories.

### Airport Dataset Overview

Data Catalog:
* **type** = airport type.
* **name** = name of the airport.
* **elevation_ft** = elevation of the airport related to the ocean.
* **continent** = continent where the airport belongs.
* **iso_country** = country iso code.
* **iso_region** = iso region code.
* **municipality** = municipality name where the airport belongs.
* **gps_code** = gps code of the airport.
* **iata_code** = a three-letter geocode designating many airports and metropolitan areas around the world.
* **local_code** = local code of the airport related to the country it belongs.
* **coordinates** = coordinates where the airport it is.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

* Clearly state the rationale for the choice of tools and technologies for the project.

The technology I chose is because I wanted to have all in mind if there will be heavy loads in the future. A data lake with S3 bucket to have all the versioning for the data we use and process.. Like an staging folder to have raw data and then a folder to have the data cleaned and ready to load it to A database or datawarehouse in parquet format.

* Propose how often the data should be updated and why.

The data will be updated monthly because the dataset is updated every month. Looking at the page there is a bulletpoint that they update the data monthly _" Summary data (selected high-volume markets) posted monthly to trade.gov"_

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x. : **_We'll need to use AWS EMR Cluster with as many nodes as we need to process the data properly because we use auto-scalling to meet heavy load requirements. Also setting caching and repartition to process the data 10x faster_**
 * The data populates a dashboard that must be updated on a daily basis by 7am every day. : **_We can use Apache Airflow with Custom Operators to execute the workflow and update the necessary tables from AWS Redshift_**
 * The database needed to be accessed by 100+ people. : **_We'll manage that in AWS Redshift (Data Warehouse). It can manage concurrency as well_**
